In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# Preparation stuff

## Connect to Drive

In [2]:
connect_to_drive = False

In [3]:
#Run command and authorize by popup --> other window
if connect_to_drive:
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)

## Install packages

In [4]:
if connect_to_drive:
    #Install FS code
    !pip install  --upgrade --force-reinstall git+https://github.com/federicosiciliano/easy_lightning.git

    !pip install pytorch_lightning

## IMPORTS

In [5]:
#Put all imports here
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
#import pickle
import os
import sys
#import cv2
import csv
import torch

## Define paths

In [6]:
#every path should start from the project folder:
project_folder = "../"
if connect_to_drive:
    project_folder = "/content/gdrive/Shareddrives/<SharedDriveName>" #Name of SharedDrive folder
    #project_folder = "/content/gdrive/MyDrive/<MyDriveName>" #Name of MyDrive folder

#Config folder should contain hyperparameters configurations
cfg_folder = os.path.join(project_folder,"cfg")

#Data folder should contain raw and preprocessed data
data_folder = os.path.join(project_folder,"data")
raw_data_folder = os.path.join(data_folder,"raw")
processed_data_folder = os.path.join(data_folder,"processed")

#Source folder should contain all the (essential) source code
source_folder = os.path.join(project_folder,"src")

#The out folder should contain all outputs: models, results, plots, etc.
out_folder = os.path.join(project_folder,"out")
img_folder = os.path.join(out_folder,"img")

## Import own code

In [7]:
#To import from src:

#attach the source folder to the start of sys.path
sys.path.insert(0, project_folder)

#import from src directory
# from src import ??? as additional_module
import easy_rec as additional_module #REMOVE THIS LINE IF IMPORTING OWN ADDITIONAL MODULE

import easy_exp, easy_rec, easy_torch #easy_data

/home/filippobetello/.conda/envs/recsys/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-21 17:40:12,878	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-21 17:40:12,961	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


# MAIN

## Train

### Data

In [8]:
cfg = easy_exp.cfg.load_configuration("config_rec")

In [9]:
#cfg["data_params"]["test_sizes"] = [cfg["data_params.dataset_params.out_seq_len.val"],cfg["data_params.dataset_params.out_seq_len.test"]]

data_params = cfg["data_params"].copy()
data_params["data_folder"] = raw_data_folder

data, maps = easy_rec.data_generation_utils.preprocess_dataset(**data_params)

Ratings data already exists. Skip pre-processing
Filtering by minimum number of users per item: 5
Filtering by minimum number of items per user: 5
Densifying index
Splitting: leave_n_out


In [10]:
# #Save user and item mappings
# # TODO: check
# with open(os.path.join(processed_data_folder,"user_map.csv"), "w") as f_user:
#     w = csv.writer(f_user)
#     w.writerows(maps['uid'].items())

# with open(os.path.join(processed_data_folder,"item_map.csv"), "w") as f_item:
#     w = csv.writer(f_item)
#     w.writerows(maps['sid'].items())

In [11]:
datasets = easy_rec.rec_torch.prepare_rec_datasets(data,**data_params["dataset_params"])

In [12]:
collator_params = cfg["data_params"]["collator_params"].copy()
collator_params["num_items"] = np.max(list(maps["sid"].values()))

In [13]:
# app = collator_params.get("negatives_distribution",None)
# if app is not None:
#     if app == "popularity":
#         collator_params["negatives_distribution"] = easy_rec.data_generation_utils.get_popularity_items(datasets["train"], collator_params["num_items"])
#     elif app not in ["uniform","dynamic"]:
#         raise ValueError("Invalid negatives distribution")

In [14]:
collators = easy_rec.rec_torch.prepare_rec_collators(**collator_params)

In [15]:
loader_params = cfg["model"]["loader_params"].copy()
loaders = easy_rec.rec_torch.prepare_rec_data_loaders(datasets, **loader_params, collate_fn=collators)

In [16]:
rec_model_params = cfg["model"]["rec_model"].copy()
rec_model_params["num_items"] = np.max(list(maps["sid"].values()))
rec_model_params["num_users"] = np.max(list(maps["uid"].values()))
rec_model_params["lookback"] = data_params["collator_params"]["lookback"]

In [17]:
main_module = easy_rec.rec_torch.create_rec_model(**rec_model_params)#, graph=easy_rec.data_generation_utils.get_graph_representation(data["train_sid"]))

Seed set to 42
/home/filippobetello/.conda/envs/recsys/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


In [18]:
exp_found, experiment_id = easy_exp.exp.get_set_experiment_id(cfg)
print("Experiment already found:", exp_found, "----> The experiment id is:", experiment_id)

Experiment already found: True ----> The experiment id is: RKbhvzXsGBaRfKyc


In [19]:
# # Find "original" implementation:
# # ...

# keys_to_change = {"model.rec_model.seed": 42}
# orig_cfg = deepcopy(cfg)
# for k,v in keys_to_change.items():
#     orig_cfg[k] = 42

# orig_exp_found, orig_experiment_id = easy_exp.exp.get_experiment_id(orig_cfg)
# print("Experiment already found:", orig_exp_found, "----> The experiment id is:", orig_experiment_id)

# Caricare modello originale (last o best) e fare predizione...
# Mettere la predizione dentro metrica RLS... --> prossime celle

In [20]:
#if exp_found: exit() #TODO: make the notebook/script stop here if the experiment is already found

In [21]:
model_params = cfg["model"].copy()

trainer_params = easy_torch.preparation.prepare_experiment_id(model_params["trainer_params"], experiment_id)

#dynamic_negatives_index = [i for i, x in enumerate(trainer_params["callbacks"]) if "DynamicNegatives" in x][0]
#trainer_params["callbacks"][dynamic_negatives_index]["DynamicNegatives"]["dataloader"] = loaders["train"]

# Prepare callbacks and logger using the prepared trainer_params
trainer_params["callbacks"] = easy_torch.preparation.prepare_callbacks(trainer_params, additional_module.callbacks)
trainer_params["logger"] = easy_torch.preparation.prepare_logger(trainer_params)

# Prepare the trainer using the prepared trainer_params
trainer = easy_torch.preparation.prepare_trainer(**trainer_params)

model_params["loss"] = easy_torch.preparation.prepare_loss(model_params["loss"], additional_module.losses)

# Prepare the optimizer using configuration from cfg
model_params["optimizer"] = easy_torch.preparation.prepare_optimizer(**model_params["optimizer"])

# Prepare the metrics using configuration from cfg
# num_negatives = {split_name:[x] for split_name,x in data_params["collator_params"]["num_negatives"].items()}
# num_negatives["val"] += num_negatives["test"] #cause using test as val just to get metrics
# model_params["metrics"] = additional_module.metrics.prepare_rank_corrections(model_params["metrics"], num_negatives = num_negatives, num_items = rec_model_params["num_items"])
model_params["metrics"] = easy_torch.preparation.prepare_metrics(model_params["metrics"], additional_module.metrics)

# Create the model using main_module, loss, and optimizer
model = easy_torch.process.create_model(main_module, **model_params)

Seed set to 42
Seed set to 42
Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42


In [22]:
# Prepare the emission tracker using configuration from cfg
#tracker = easy_torch.preparation.prepare_emission_tracker(**cfg["model"]["emission_tracker"], experiment_id=experiment_id)

In [23]:
# Prepare the flops profiler using configuration from cfg
#profiler = easy_torch.preparation.prepare_flops_profiler(model=model, **cfg["model"]["flops_profiler"], experiment_id=experiment_id)

### Train

In [24]:
#easy_torch.process.test_model(trainer, model, loaders, test_key=["train","val","test"]) #, tracker=tracker, profiler=profiler)

In [25]:
# Train the model using the prepared trainer, model, and data loaders
import time
start = time.time()
easy_torch.process.train_model(trainer, model, loaders, val_key=["val","test"]) #tracker=tracker, profiler=profiler, 
print("Elapsed time:", time.time()-start)

Seed set to 42
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/filippobetello/.conda/envs/recsys/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /hdd2/filippobetello/projects/environmental_sampling/out/models/prov/RKbhvzXsGBaRfKyc exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name        | Type                        | Params | Mode 
--------------------------------------------------------------------
0 | main_module | SASRec                      | 149 K  | train
1 | loss        | SequentialBCEWithLogitsLoss | 0      | train
2 | metrics     | RobustModuleDict            | 0      | train
----------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/filippobetello/.conda/envs/recsys/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=27` in the `DataLoader` to improve performance.


/home/filippobetello/.conda/envs/recsys/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=27` in the `DataLoader` to improve performance.
/home/filippobetello/.conda/envs/recsys/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (8) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 299: 100%|██████████| 8/8 [00:01<00:00,  5.66it/s, v_num=1]

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 299: 100%|██████████| 8/8 [00:01<00:00,  5.53it/s, v_num=1]
Elapsed time: 409.2964172363281


In [26]:
easy_torch.process.test_model(trainer, model, loaders) #, tracker=tracker, profiler=profiler)

Seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]
/home/filippobetello/.conda/envs/recsys/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=27` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 8/8 [00:00<00:00, 29.61it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        test_F1_@10        │    0.11343970149755478    │
│        test_F1_@20        │    0.0745747983455658     │
│        test_F1_@5         │    0.14800255000591278    │
│       test_MAP_@10        │    0.09136956930160522    │
│       test_MAP_@20        │    0.06912802904844284    │
│        test_MAP_@5        │    0.11232944577932358    │
│       test_MRR_@10        │    0.2740388512611389     │
│       test_MRR_@20        │    0.2849952280521393     │
│        test_MRR_@5        │    0.24946977198123932    │
│       test_NDCG_@10       │    0.3542914092540741     │
│       test_NDCG_@20       │    0.3947583734989166     │
│       test_NDCG_@5        │    0.2948862314224243     │
│    test_Precision_@10     │   0.061717912554740906    │
│    test_Precision_@20     │    0.03891834616661072    │
│     test_Precision_@5     │    0.08674443513154984    │
│      test_Recall_@10      │    0.6171792149543762     │
│      test_Recall_@20      │    0.7783669233322144     │
│      test_Recall_@5       │    0.4337221682071686     │
│         test_loss         │    0.40069133043289185    │
└───────────────────────────┴───────────────────────────┘

In [27]:
        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        test_F1_@10        │    0.09528966248035431    │
│        test_F1_@20        │    0.0691293403506279     │
│        test_F1_@5         │    0.11542344838380814    │
│       test_MAP_@10        │    0.07273733615875244    │
│       test_MAP_@20        │    0.0568661242723465     │
│        test_MAP_@5        │    0.08753623068332672    │
│       test_MRR_@10        │    0.2184016853570938     │
│       test_MRR_@20        │    0.23114466667175293    │
│        test_MRR_@5        │    0.19386708736419678    │
│       test_NDCG_@10       │    0.28817281126976013    │
│       test_NDCG_@20       │    0.3358105421066284     │
│       test_NDCG_@5        │    0.2289959192276001     │
│    test_Precision_@10     │    0.05185578390955925    │
│    test_Precision_@20     │   0.035471901297569275    │
│     test_Precision_@5     │    0.06723223626613617    │
│      test_Recall_@10      │    0.5185577869415283     │
│      test_Recall_@20      │    0.7094379663467407     │
│      test_Recall_@5       │    0.33616119623184204    │
│         test_loss         │    0.40365272760391235    │
└───────────────────────────┴─────────────────────────

SyntaxError: invalid character '┃' (U+2503) (828045273.py, line 1)

In [ ]:
# Save experiment and print the current configuration
#save_experiment_and_print_config(cfg)
easy_exp.exp.save_experiment(cfg)

# Print completion message
print("Execution completed.")
print("######################################################################")
print()

Execution completed.
######################################################################

